In [35]:
import numpy as np
import phyre
import os
import pathlib
import cv2
import matplotlib.pyplot as plt
import random
import math
from sys import exit
import h5py
import pygame
from pygame.locals import *
import pickle

In [36]:
#print('All eval setups:', *phyre.MAIN_EVAL_SETUPS)
# For now, let's select cross template for ball tier.
eval_setup = 'ball_within_template'

fold_id = 0  # For simplicity, we will just use one fold for evaluation.
train_tasks, dev_tasks, test_tasks = phyre.get_fold(eval_setup, fold_id)

all_tasks = sorted(train_tasks+dev_tasks+test_tasks)
task2 = all_tasks[200:300]
action_tier = phyre.eval_setup_to_action_tier(eval_setup)

In [37]:
def line_coor(arr, ln):
    
    length = arr[3]*ln
    
    x1 = arr[0]*ln - (length/2)
    y1 = ln - (arr[1]*ln)

    x2 = arr[0]*ln + (length/2)
    y2 = ln - (arr[1]*ln)

    return [x1, y1], [x2, y2]

In [38]:
def ball_coor(arr, ln):
    
    x1 = arr[0]*ln
    y1 = ln - arr[1]*ln
    
    rad = (arr[3]*ln)/2.
    
    return [x1,y1], rad

In [39]:
def create_traj(f_list, iter_path):
    
    traj_path = iter_path+"/"+'traj.jpg'
    
    ln = 256
    
    pygame.init()
    
    window = pygame.display.set_mode((ln, ln))


    window.fill((255, 255, 255))

    c1, c2 = line_coor(f_list[0][0], ln)
    pygame.draw.line(window, (0,0,0),c1, c2,7)
    
    black_bar = [c1, c2]

    c3, c4 = line_coor(f_list[0][2], ln)
    pygame.draw.line(window, (0,0,255),c3, c4,10)

    target_bar = [c3,c4]
    
    red_coords = []
    green_coords = []
    
    for i in range(f_list.shape[0]):

        c_gr, r_gr = ball_coor(f_list[i][1],ln)
        green_coords.append(c_gr)

        pygame.draw.circle(window, (0,255,0), c_gr, r_gr, 0)

        c_rd, r_rd = ball_coor(f_list[i][3],ln)
        red_coords.append(c_rd)
        
        pygame.draw.circle(window, (255,0,0), c_rd, r_rd, 0)
    

    pygame.display.update()
    pygame.image.save(window, traj_path)
    pygame.quit()
    
    h5path = iter_path + "/" + "Coordinates.h5"
    
    with h5py.File(h5path, 'w') as hf:
        hf.create_dataset('green_coords', data=green_coords)
        hf.create_dataset('green_radius', data=[r_gr])
        hf.create_dataset('red_coords', data=red_coords)
        hf.create_dataset('red_radius', data=[r_rd])
        hf.create_dataset('black_coords', data=black_bar)
        hf.create_dataset('target_coords', data=target_bar)

In [41]:
parent_dir = 'traj_test'
taskid = '00002'
task_path = parent_dir + '/' + taskid

pathlib.Path(task_path).mkdir(parents = True, exist_ok = True)
req_sim = 10

for episode in task2:
    
    simulator = phyre.initialize_simulator([episode], action_tier)    
    sims = 0
    ep_path = task_path + '/' + str(episode)
    pathlib.Path(ep_path).mkdir(parents = True, exist_ok = True)

    
    while sims < req_sim:
        
        action = simulator.sample()
        simulation = simulator.simulate_action(0, action, need_images=True, need_featurized_objects=True)
        
        if simulation.status.is_solved():
            
            sims += 1
            iter_path = ep_path + '/' + str(sims)
            pathlib.Path(iter_path).mkdir(parents = True, exist_ok = True)

            for idx, frame in enumerate(simulation.images):
                
                img = phyre.observations_to_uint8_rgb(frame)
                img_path = iter_path+'/'+str(idx+1)+'.jpg'
                cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
                
            f_path = iter_path+"/"+"feature_list.pkl"
            
            with open(f_path, 'wb') as handle:
                pickle.dump(simulation.featurized_objects, handle, protocol=pickle.HIGHEST_PROTOCOL)     
                
            f_list = simulation.featurized_objects.features          
            create_traj(f_list, iter_path)

In [25]:
with h5py.File('/home/mihir/traj_test/00002/00002:197/10/Coordinates.h5') as hf:
    
    keys = list(hf.keys())
    green_coords = np.array(hf['green_coords'])
    black_coords = np.array(hf['black_coords'])

In [26]:
keys

['black_coords',
 'green_coords',
 'green_radius',
 'red_coords',
 'red_radius',
 'target_coords']

In [28]:
green_coords

array([[166.3999939 ,  13.11633301],
       [166.3999939 ,  43.98634338],
       [155.82922363,  82.89988708],
       [141.96551514,  84.72462463],
       [128.14953613,  84.72462463],
       [114.3795166 ,  84.72462463],
       [100.64568329,  84.84422302],
       [ 85.08218384, 104.99078369],
       [ 69.08094025, 181.37836456],
       [ 52.46472931, 232.44414902],
       [ 35.32846832, 238.32358551],
       [ 18.26213455, 238.32358551],
       [ 13.40760899, 238.32276917],
       [ 13.77805138, 238.32276917],
       [ 14.14726257, 238.32276917]])

In [29]:
black_coords

array([[102.34881592,  99.83999634],
       [230.45117188,  99.83999634]])